In [15]:
%%html
<style>
    table{
        width:70%;
    }
</style>

## EM algrithom

### 原理

EM算法是存在隐变量下的极大似然估计
$$ MLE  \xrightarrow[]{\text{latent variable } Z}  EM  $$ 

E step: 
$$ E_{Z|Y,\theta^{(t)}} [P(Y,Z|\theta] $$

M step:
$$ \theta^{(t+1)} = \arg\max_{\theta}  E_{Z|Y,\theta^{(t)}} [ \log P(Y,Z | \theta) ] $$

### 推导

似然函数 $L(\theta)$ 为（考虑隐变量$Z$的）Y的极大似然估计

$$ 
\begin{align}
L(\theta) =& \sum_{i=1}^N \log P(Y | \theta) \\
          =& \sum_{i=1}^N \log \sum_{z} P(Y, Z | \theta) \\
          =& \sum_{i=1}^N \log \sum_{z} P(Z|Y,\theta^{(t)}) \frac{P(Y, Z | \theta)}{P(Z|Y,\theta^{(t)})} \\
          \geq & \sum_{i=1}^N \sum_{z} P(Z|Y,\theta^{(t)}) \log \frac{P(Y, Z | \theta)}{P(Z|Y,\theta^{(t)})} \\
\end{align}
$$

- 不等号是詹森不等式 Jensen Inequality 的应用
- 第3，4个等号，同时乘以隐变量在【当前模型参数$\theta^{(t)}$ 和 观测$Y$】下的条件概率分布 $P(Z|Y,\theta^{(t)})$
- 当Z为常量时取等号，ie. $P(Z_z|Y_i,\theta^\star) \rightarrow 1$ 随着迭代过程的演进，每个样本的隐变量取值趋于确定 【个人理解，待验证】

因为我们的目的是最大化似然函数，抛开常数项，似然函数下界的最大化问题为

$$ \max_{\theta} \sum_{i=1}^N \sum_{z} P(Z|Y,\theta^{(t)}) \log P(Y, Z | \theta) $$

上式中最大化的目标函数也称为Q函数。外层求和号里面的公式可以写为 $$ E_{Z|Y,\theta^{(t)}} \log P(Y,Z|\theta) $$ 即EM算法名字中的E步。

M步，即（在当前模型基础上）极大化Q函数:

1. 根据模型的具体形式写出 $P(Z|Y,\theta), P(Y,Z|\theta)$ 的具体形式
2. 对$\theta$各分量求偏导数，令其为0，解出迭代公式

### 三硬币模型

#### Notations

参数： $$ \theta = \begin{bmatrix} P_A \\ P_B \\ \pi \\ \end{bmatrix} \\ $$

条件概率 P(Z|Y)：
$$ 
\begin{align}
\hat \gamma_{Ai}^{(t)} &\overset{\triangle}{=} P(Z_i=A|Y_i,\theta^{(t)})   \\
\hat \gamma_{Bi}^{(t)} &\overset{\triangle}{=} P(Z_i=B|Y_i,\theta^{(t)}) = 1 - \hat\gamma_{Ai}^{(t)}
\end{align}
$$

完全概率 P(Y,Z)

| Complete Prob P(Y,Z) | Y=1           | Y=0               | Compact form                  |
|----------------------|---------------|-------------------|-------------------------------|
| Z=coinA              | $\pi P_A$     | $\pi (1-P_A)$     | $\pi P_A^y (1-P_A)^{1-y}$     |
| Z=coinB              | $(1-\pi) P_B$ | $(1-\pi) (1-P_B)$ | $(1-\pi) P_B^y (1-P_B)^{1-y}$ |

#### 迭代公式的具体形式推导

注意，当前模型下的条件概率 $\hat\gamma_{Ai}^{(t)}, \hat\gamma_{Bi}^{(t)}$ 都是可计算的数字，完全概率查上表可得

写出三硬币模型对数似然函数的具体形式：

$$ \begin{align}
L(\theta) &= \sum_{i=1}^N \sum_z P(Z|Y,\theta^{(t)}) \log P(Y,Z|\theta)  \\
          &= \sum_{i=1}^N \hat\gamma_{Ai}^{(t)} \boxed{ \log P(Y,Z|\theta) } + 
                          \hat\gamma_{Bi}^{(t)} \boxed{ \log P(Y,Z|\theta) } \\
          &= \sum_{i=1}^N \hat\gamma_{Ai}^{(t)} \boxed{ \log[\pi P_A^y(1-P_A)^{1-y}] } + 
                          \hat\gamma_{Bi}^{(t)} \boxed{ \log[(1-\pi)P_B^y(1-P_B)^{1-y}] } \\
          &= \sum_{i=1}^N \hat\gamma_{Ai}^{(t)} \boxed{ \log\pi + y\log P_A + (1-y)\log(1-P_A)) }+ 
                          \hat\gamma_{Bi}^{(t)} \boxed{ \log(1-\pi) + y\log P_B + (1-y)\log(1-P_B)) }\\
\end{align}$$ 

求偏导数，令其为零：

$$
\begin{align}
\frac{\partial L}{\partial \pi} &= 
                \sum_{i=1}^N \hat\gamma_{Ai}^{(t)} \frac{1}{\pi} + \hat\gamma_{Bi}^{(t)} \frac{1}{1-\pi} &=0 \\
\frac{\partial L}{\partial P_A} &= \sum_{i=1}^N \hat\gamma_{Ai}^{(t)} ( \frac{y_i}{P_A} - \frac{1-y_i}{1-P_A} ) &=0\\
\frac{\partial L}{\partial P_B} &= \sum_{i=1}^N \hat\gamma_{Bi}^{(t)} ( \frac{y_i}{P_B} - \frac{1-y_i}{1-P_B} ) &=0\\
\end{align}
$$ 

简单的整理，得到解
$$
\begin{align}
\pi &= \frac{\sum_{i=1}^{N} \hat\gamma_{Ai}^{(t)}}{N} \\
P_A &= \frac{\sum_{i=1}^{N} \hat\gamma_{Ai}^{(t)} y_i}{ \sum_{i=1}^{N} \hat\gamma_{Ai}^{(t)} } \\
P_B &= \frac{\sum_{i=1}^{N} \hat\gamma_{Bi}^{(t)} y_i}{ \sum_{i=1}^{N} \hat\gamma_{Bi}^{(t)} } = 
  \frac{\sum_{i=1}^{N} (1-\hat\gamma_{Ai}^{(t)}) y_i}{ \sum_{i=1}^{N} (1-\hat\gamma_{Ai}^{(t)}) } \\
\end{align}
$$

回忆一下 $\hat\gamma_{A_i}^{(t)}$的含义：它是指在$t$次迭代的模型中，根据每个样本观测值$Y_i$，样本$i$的隐变量$Z_i$为硬币A的条件概率。

根据条件概率公式：
$$ 
\hat\gamma_{A_i}^{(t)} = P(Z_i=A|Y_i,\theta^{(t)}) = \frac{P(Z_i=A,Y_i|\theta^{(t)})}{P(Y_i|\theta^{(t)})} = 
\begin{cases}
\frac{\pi P_A^{(t)}}{\pi^{(t)} P_A^{(t)} + \big(1-\pi^{(t)}\big) P_B^{(t)}}, & Y_i = 1 \\
\frac{\pi \big(1-P_A^{(t)}\big)}{\pi^{(t)} \big(1-P_A^{(t)}\big) + \big(1-\pi^{(t)}\big)\big(1-P_B^{(t)}\big)}, & Y_i = 0 
\end{cases}
$$

在每一轮迭代中，会计算出每个样本的 $\hat\gamma_{A_i}^{(t)}$, $\hat\gamma_{B_i}^{(t)}$，观察发现他们应该是逐渐趋于真实的$\gamma_i$。

#### 三硬币模型 Code

In [224]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline


# Data Generating Process
N = 10000
pi = 0.3
pa = 0.9
pb = 0.1

df = pd.DataFrame({'Z': np.random.choice(['A', 'B'], N)})
s = df.groupby('Z').apply(lambda df: np.random.choice([1,0], len(df), p=[pa,1-pa] if df['Z'].unique() == 'A' else [pb,1-pb]))
df.loc[df.Z == 'A', 'Y'] = s['A']
df.loc[df.Z == 'B', 'Y'] = s['B']
df['Y'] = df['Y'].astype(int)

#df.head()

In [225]:
def CondP_A(y, pi, pa, pb):
    """条件后验概率函数 P(Z=A|Y)
    
    y: array.
    pi, pa, pb: scalar
    returns: array
    """
    N = len(y)
    pA1 = _CondP_1(pi, pa, pb)
    pA0 = _CondP_0(pi, pa, pb)
    return np.repeat(pA1, N) * y + np.repeat(pA0, N) * (1-y)

def _CondP_1(pi, pa, pb):
    """P(Z=A|Y=1)"""
    return pi*pa /(pi * pa + (1-pi) * pb)

def _CondP_0(pi, pa, pb):
    """P(Z=A|Y=0)"""
    return pi*(1-pa) /(pi * (1-pa) + (1-pi) * (1-pb))

df['CondProb_A'] = CondP_A(df['Y'], pi, pa, pb)
df['CondProb_B'] = 1 - df['CondProb_A']
#df.head()

In [226]:
def updated_param(condprobA, y):
    """更新参数 theta = [PA, PB, pi]
    
    *发现参数迭代公式中只有条件概率P(Z|Y)，而完全概率P(Y,Z)由于具有具体形式，在求偏导数的过程中已经用掉了*
    """
    pi = np.average(condprobA)
    pa = np.average(y, weights=condprobA)
    pb = np.average(y, weights=1-condprobA)
    return pi, pa, pb

#updated_param(df['CondProb_A'], df['Y'])

In [227]:
n_iter = 10

y = df['Y'].values
theta = 0.5, 0.5, 0.5

print('true value:')
print((pi, pa, pb), 'P(Z=A|Y=1):', _CondP_1(pi, pa,pb), 'P(Z=A|Y=0):', _CondP_0(pi, pa, pb))
print("P(Y=1)", pi * pa + (1-pi)*pb)
print('----------------------')

for i in range(n_iter):
    pi, pa, pb = theta
    PY1 = pi * pa + (1-pi)*pb
    PZ0 = _CondP_0(*theta)
    PZ1 = _CondP_1(*theta)
    print(theta, 'P(Z=A|Y=1):', PZ1, 'P(Z=A|Y=0):', PZ0, 'P(Y=1):', PY1)
    condprobA = CondP_A(y, *theta)
    theta = updated_param(condprobA, y)

true value:
(0.3, 0.9, 0.1) P(Z=A|Y=1): 0.7941176470588235 P(Z=A|Y=0): 0.04545454545454544
P(Y=1) 0.34
----------------------
(0.5, 0.5, 0.5) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.5
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973
(0.5, 0.4973, 0.4973) P(Z=A|Y=1): 0.5 P(Z=A|Y=0): 0.5 P(Y=1): 0.4973


#### 三硬币模型：结论

尝试各种真实参数、初始参数，结果发现几乎100%会陷入错误的局部最优值。且无视隐变量、只看最终结果的概率 P(Y=1) 也不一定能正确估计。

直觉上很好理解：这个模型本身就难通过一个0-1变量将Z的概率剥离出来，每个样本只有一次观测，当你仅有一个观测样本Y=1，很难说究竟是因为pi很大还是因为 P_A or P_B 比较大。

- 拓展的三硬币模型：见 *What's EM ?*, Chuong B Do & Serafim Batzoglou
- 高斯混合模型